<a href="https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/U01/MyFirstKerasNN-PimaIndian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="Bilder/ost_logo.png" width="240" height="120" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2022 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>


## Ein erstes Neuronales Netz mit `keras`: Pima Indians

## (a) Datensatz laden

In [ ]:
# first neural network with keras tutorial
import tensorflow as tf
import numpy as np
from numpy import loadtxt

import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


import matplotlib.pyplot as plt


In [ ]:
# load the dataset
# dataset = loadtxt('pima-indians-diabetes.csv', delimiter=',')
# split into input (X) and output (y) variables
#df=pd.read_csv('diabetes.csv', delimiter=',')
df=pd.read_csv('https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/U01/diabetes.csv',delimiter=',')
df.head()


## (b) Kurze EDA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool_)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))


# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, vmin=0.0, center=0, annot=True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});
plt.show()


In [ ]:
df.describe()

In [ ]:
X = df.iloc[:,0:8].values
y = df.iloc[:,8].values


## (c) Modell erstellen

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

## (d) Modell darstellen

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

## (e) Modell kompilieren und trainieren

In [ ]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# fit the keras model on the dataset
model.fit(X, y, epochs=100, batch_size=10)
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

## (f) Modell abspeichern

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Sie können Ihr Modell speichern, indem Sie die Funktion `save()` für das Modell aufrufen und den Dateinamen angeben.
Das folgende Beispiel demonstriert dies, indem zunächst ein Modell angepasst, ausgewertet und in der Datei "model.h5" gespeichert wird.

In [ ]:
# save model and architecture to single file
model.save("model.h5")
print("Saved model to disk")

## (g) Modell laden

In [ ]:
# load and evaluate a saved model

from keras.models import load_model
from keras.models import model_from_json

# load model
model = load_model('model.h5')
# summarize model.
model.summary()

# load dataset
df=pd.read_csv('https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/U01/diabetes.csv',
               delimiter=',')

# split into input (X) and output (Y) variables
X = df.iloc[:,0:8].values
y = df.iloc[:,8].values

# evaluate the model
score = model.evaluate(X, y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

## (h) Aufteilen in einen Trainings und Validierungsdatensatz

Teilen Sie den geladenen Datensatz in einen Trainings- und einen Testdatensatz auf (Aufteilung nach Zeilen) und verwenden Sie einen die Trainingsdaten (80%), um das Modell zu trainieren, und den Validierungsdatensatz (20%), um die Leistung des Modells auf neuen Daten zu schätzen.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

val_dataframe   = df.sample(frac=0.2, random_state=1337)
train_dataframe = df.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)


## (i) Dataset-Objekt erstellen

Die `tf.data`-API (\url{https://www.tensorflow.org/guide/data}) ermöglicht es Ihnen, komplexe Eingangs-Pipelines aus einfachen, wiederverwendbaren Teilen aufzubauen. Die Pipeline für ein Bildmodell könnte beispielsweise Daten aus Dateien in einem verteilten Dateisystem aggregieren, zufällige Störungen auf jedes Bild anwenden und zufällig ausgewählte Bilder zum Training zu einem Stapel (batch) zusammenführen. Die Pipeline für Texdaten kann das Extrahieren von Symbolen aus Rohtextdaten, deren Umwandlung in ein Embeddding und das Zusammenführen von Sequenzen unterschiedlicher Länge umfassen. Die `tf.data` API ermöglicht es , grosse Datenmengen zu verarbeiten, liest aus verschiedenen Datenformaten und ist in der Lage, komplexe Transformationen durchzuführen.
		
Die `tf.data`-API führt ein \texttt{tf.data.Dataset}-Objekt ein, welches eine Sequenz von Elementen darstellt, wobei jedes Element aus einer oder mehreren Komponenten besteht. In einer Bild-Pipeline könnte ein Element zum Beispiel ein einzelnes Trainingsbeispiel sein, bestehend aus einem Tupel aus je zwei Tensoren, die das Bild und sein Label darstellen.


In [ ]:
def dataframe_to_dataset(df):
    dg = df.copy()
    labels = dg.pop('Outcome')
    dataset = tf.data.Dataset.from_tensor_slices((dg.values, labels.values))
    dataset = dataset.batch(32).repeat()
    dataset = dataset.shuffle(buffer_size=len(df))
    return dataset

In [ ]:
train_dataset=dataframe_to_dataset(train_dataframe)
val_dataset  =dataframe_to_dataset(val_dataframe)

## (j) Lernkurven grafisch darstellen

Die Funktion `fit()` gibt ein `History`-Objekt zurück, das den Verlust und die Genauigkeit am Ende jeder Epoche zusammenfasst. Erstellen Sie Liniendiagramme dieser Daten, sogenannte Lernkurven.


In [ ]:
def plot_history(history):
    #print(history.history.keys())
    #  "Accuracy"
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.grid(True); plt.show()
    # "Loss"
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.grid(True); plt.show()

In [ ]:
history=model.fit(train_dataset, epochs=50, 
          steps_per_epoch=100,
          validation_data=val_dataset,
          validation_steps=10)

print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
plot_history(history)

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(16, input_dim=8, activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history=model.fit(train_dataset, epochs=50, 
          steps_per_epoch=100,
          validation_data=val_dataset,
          validation_steps=10)

print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
plot_history(history)

## (l) Trainieren auf einem neuen Datensatzes

Probieren Sie, einen anderen Tabellendatensatz zu verwenden, vielleicht aus dem UCI Machine Learning Repository.

## (k)  Das Modell anpassen

Ändern Sie die Konfiguration des Modells oder den Trainingsprozess und sehen Sie, ob Sie die Leistung des Modells verbessern können, z. B. eine Genauigkeit von mehr als 76 % erreichen.
